### High-Performance Pandas: eval() and query()

#### Motivating query() and eval(): Compound Expressions

In [1]:
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(1E6)
y = rng.rand(1E6)

%timeit x + y

C:\Users\Aroogz\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()
C:\Users\Aroogz\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


100 loops, best of 3: 5.01 ms per loop


In [2]:
# using base Python
%timeit np.fromiter?

In [8]:
%timeit np.fromiter([xi + yi for xi, yi in zip(x,y)], dtype=x.dtype, count=len(x))

1 loop, best of 3: 245 ms per loop


But this abstraction can be less efficient when you are computing compound expressions. 

In [10]:
#e.g  for below expression:
mask = (x > 0.5) & (y < 0.5)

# it is equivalent to below as each subexpression is roughly equivalent
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

so thus is inefficient as every intermediate step is allocated in memory. The numexpr package is a way out of this

In [15]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask , mask_numexpr)

True

#### Pandas.eval() for Efficient Operations

In [17]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols)) 
                     for i in range(4))

In [22]:
%timeit df1 + df2 + df3 + df4

10 loops, best of 3: 155 ms per loop


In [23]:
%timeit pd.eval('df1 + df2 + df3 + df4')

10 loops, best of 3: 72 ms per loop


__About 50% faster!__

##### Operations supported by pd.eval()

In [24]:
df1,df2,df3,df4,df5 = (pd.DataFrame(rng.randint(0,100,(100,3)))
                      for i in range(5))

##### Arithmetic operators

Supports all arithmetic operators 

In [26]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

##### Comparison operators
including chains

In [27]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

##### Bitwise operators
`&` and `|`

In [29]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

and in addition, it supports leteral and Boolean expressions

In [30]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

##### Object attributes and indices

In [32]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

__other operations such as function calls are currently not implemented in Pd.eval(). Numexpr library can be imported for more functionalities__

#### DataFrame.eval() for column-wise operations

In [33]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.871498,0.765464,0.971125
1,0.332477,0.170807,0.105295
2,0.934971,0.417575,0.371057
3,0.671115,0.153091,0.246334
4,0.093113,0.026666,0.150374


In [35]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B)/(df.C - 1)")
np.allclose(result1, result2)

True

In [36]:
# or more succintly
result3 = df.eval('(A+B)/(C-1)')
np.allclose(result1, result3)

True

##### Assignment

In [37]:
df.head()

,A,B,C
0,0.871498,0.765464,0.971125
1,0.332477,0.170807,0.105295
2,0.934971,0.417575,0.371057
3,0.671115,0.153091,0.246334
4,0.093113,0.026666,0.150374


In [40]:
df.eval('D = (A+B)/C', inplace=1)
df.head()

,A,B,C,D
0,0.871498,0.765464,0.971125,1.685635
1,0.332477,0.170807,0.105295,4.779742
2,0.934971,0.417575,0.371057,3.645121
3,0.671115,0.153091,0.246334,3.345893
4,0.093113,0.026666,0.150374,0.796539


##### Local variables in DataFrame.eval()

In [45]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean') # @ gives access to another namespace
# this is only available for the df.eval()

np.allclose(result1, result2)

True

#### DataFramr.query() Method

In [49]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

We can't use the df.eval() s the dataframe is not available in the namespace of df itself. Instead we do the following

In [51]:
result = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

In [53]:
# with local variables:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

__*In general these fucntions are good for compound expressions as they avoid the memory allotment that plague traditional pandas or NumPy methods. Also they have cleaner syntax*__

[Further Details](http://pandas-docs.github.io/pandas-docs-travis/enhancingperf.html)